# Floor plan detector

In [1]:
"""
TRAINING PIPELINE FOR FLOORPLAN OBJECT DETECTION FROM SCRATCH
"""

import json
import os
import numpy as np
import random
import time
from pathlib import Path
from collections import defaultdict
import warnings

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, SequentialSampler
from torch.optim.lr_scheduler import CosineAnnealingLR

import cv2
from PIL import Image
from tqdm import tqdm
from torchvision.ops import box_iou, batched_nms

warnings.filterwarnings('ignore')

In [2]:
CONFIG = {
    'dataset': {
        'image_dir': '/content/drive/MyDrive/assets/train',
        'annotation_file': '/content/drive/MyDrive/annotations.coco.json',
        'train_split': 0.80,
        'val_split': 0.10,
    },
    'training': {
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'batch_size': 4,
        'num_epochs': 50,
        'learning_rate': 1e-4,
        'weight_decay': 1e-4,
        'warmup_epochs': 5,
        'early_stopping_patience': 15,
    },
    'model': {
        'num_fg_classes': 8,
        'num_anchors': 9,
        'image_size': 640,
    }
}

In [3]:
class COCOFloorplanDataset:
    def __init__(self, image_dir, annotation_file, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        with open(annotation_file, 'r') as f:
            self.coco_data = json.load(f)
        self.images = self.coco_data['images']
        self.annotations = self.coco_data['annotations']
        self.img_to_anns = defaultdict(list)
        for ann in self.annotations:
            self.img_to_anns[ann['image_id']].append(ann)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_info = self.images[idx]
        img_id = img_info['id']
        img_path = os.path.join(self.image_dir, img_info['file_name'])
        image = Image.open(img_path).convert('RGB')
        anns = self.img_to_anns[img_id]
        boxes, labels = [], []
        for ann in anns:
            x, y, w, h = ann['bbox']
            boxes.append([x, y, x + w, y + h])
            labels.append(ann['category_id'])
        if not boxes:
            boxes = np.empty((0, 4), dtype=np.float32)
            labels = np.empty((0,), dtype=np.int64)
        else:
            boxes = np.array(boxes, dtype=np.float32)
            labels = np.array(labels, dtype=np.int64)
        targets = {'boxes': boxes, 'labels': labels, 'image_id': img_id}
        if self.transform:
            image, targets = self.transform(image, targets)
        return image, targets

def collate_fn(batch):
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    images = torch.stack(images, 0)
    return images, targets

In [4]:
class FloorplanAugmentation:
    def __init__(self, train=True, target_size=640):
        self.train = train
        self.target_size = target_size

    def __call__(self, image, targets):
        image = np.array(image)
        height, width = image.shape[:2]
        boxes = targets['boxes'].copy()

        if self.train:
            if random.random() < 0.5:
                image = cv2.flip(image, 1)
                if len(boxes) > 0:
                    boxes_copy = boxes.copy()
                    boxes[:, 0] = width - boxes_copy[:, 2]
                    boxes[:, 2] = width - boxes_copy[:, 0]

            if random.random() < 0.3:
                image = cv2.flip(image, 0)
                if len(boxes) > 0:
                    boxes_copy = boxes.copy()
                    boxes[:, 1] = height - boxes_copy[:, 3]
                    boxes[:, 3] = height - boxes_copy[:, 1]

            if random.random() < 0.5:
                brightness = random.uniform(0.8, 1.2)
                contrast = random.uniform(0.8, 1.2)
                image = cv2.convertScaleAbs(image, alpha=contrast, beta=brightness)

        scale_x = self.target_size / width
        scale_y = self.target_size / height
        image = cv2.resize(image, (self.target_size, self.target_size))

        if len(boxes) > 0:
            boxes[:, 0] *= scale_x
            boxes[:, 2] *= scale_x
            boxes[:, 1] *= scale_y
            boxes[:, 3] *= scale_y
            boxes[:, 0::2] = np.clip(boxes[:, 0::2], 0, self.target_size)
            boxes[:, 1::2] = np.clip(boxes[:, 1::2], 0, self.target_size)

        targets['boxes'] = boxes
        image = image.astype(np.float32) / 255.0
        image = torch.from_numpy(image).permute(2, 0, 1)

        targets['boxes'] = torch.as_tensor(targets['boxes'], dtype=torch.float32)
        targets['labels'] = torch.as_tensor(targets['labels'], dtype=torch.int64)
        targets['image_id'] = torch.as_tensor(targets['image_id'], dtype=torch.int64)
        if 'area' in targets:
            targets['area'] = torch.as_tensor(targets['area'], dtype=torch.float32)

        return image, targets

In [5]:
class SimpleBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(64, 64, 2)
        self.layer2 = self._make_layer(64, 128, 2, stride=2)
        self.layer3 = self._make_layer(128, 256, 2, stride=2)

    def _make_layer(self, in_channels, out_channels, blocks, stride=1):
        layers = [
            nn.Conv2d(in_channels, out_channels, 3, stride=stride, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        ]
        for _ in range(1, blocks):
            layers.append(nn.Conv2d(out_channels, out_channels, 3, padding=1))
            layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.ReLU(inplace=True))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

class ObjectDetectorFromScratch(nn.Module):
    def __init__(self, num_fg_classes=8, num_anchors=9, target_size=640):
        super().__init__()
        self.backbone = SimpleBackbone()
        self.num_fg_classes = num_fg_classes
        self.num_classes = num_fg_classes + 1
        self.num_anchors = num_anchors
        self.conv_head = nn.Conv2d(256, 256, 3, padding=1)
        self.cls_logits = nn.Conv2d(256, num_anchors * self.num_classes, 1)
        self.bbox_pred = nn.Conv2d(256, num_anchors * 4, 1)
        self.feature_map_size = 40
        self.stride = target_size / self.feature_map_size
        self.anchor_scales = [1.0, 1.5, 2.0]
        self.aspect_ratios = [0.5, 1.0, 2.0]
        self.pos_thresh = 0.5
        self.neg_thresh = 0.4
        anchors = self._generate_anchors(target_size)
        self.register_buffer("anchors", anchors)

    def _generate_anchors(self, target_size):
        scales = torch.tensor(self.anchor_scales)
        ratios = torch.tensor(self.aspect_ratios)
        grid_x = (torch.arange(0, self.feature_map_size) + 0.5) * self.stride
        grid_y = (torch.arange(0, self.feature_map_size) + 0.5) * self.stride
        y_c, x_c = torch.meshgrid(grid_y, grid_x, indexing='ij')
        centers = torch.stack([x_c, y_c], dim=-1).float().view(-1, 1, 2)
        base_anchors_wh = []
        for scale in scales:
             for ratio in ratios:
                base_size = self.stride * scale
                w = base_size * torch.sqrt(ratio)
                h = base_size / torch.sqrt(ratio)
                base_anchors_wh.append([w, h])
        base_anchors_wh = torch.tensor(base_anchors_wh).float()
        centers_wh = torch.cat([centers.expand(-1, self.num_anchors, -1),
                                base_anchors_wh.expand(self.feature_map_size**2, -1, -1)], dim=-1)
        all_anchors_ctr = centers_wh.view(-1, 4)
        x1y1 = all_anchors_ctr[:, :2] - all_anchors_ctr[:, 2:] / 2
        x2y2 = all_anchors_ctr[:, :2] + all_anchors_ctr[:, 2:] / 2
        return torch.cat([x1y1, x2y2], dim=-1).clamp(0, target_size)

    def _encode_boxes(self, anchors, gt_boxes):
        eps = 1e-6
        anchors_wh = anchors[:, 2:] - anchors[:, :2]
        anchors_ctr = anchors[:, :2] + 0.5 * anchors_wh
        gt_wh = gt_boxes[:, 2:] - gt_boxes[:, :2]
        gt_ctr = gt_boxes[:, :2] + 0.5 * gt_wh
        tx = (gt_ctr[:, 0] - anchors_ctr[:, 0]) / (anchors_wh[:, 0] + eps)
        ty = (gt_ctr[:, 1] - anchors_ctr[:, 1]) / (anchors_wh[:, 1] + eps)
        tw = torch.log((gt_wh[:, 0] / (anchors_wh[:, 0] + eps)) + eps)
        th = torch.log((gt_wh[:, 1] / (anchors_wh[:, 1] + eps)) + eps)
        return torch.stack([tx, ty, tw, th], dim=1)

    def _decode_boxes(self, anchors, deltas):
        anchors_wh = anchors[:, 2:] - anchors[:, :2]
        anchors_ctr = anchors[:, :2] + 0.5 * anchors_wh
        pred_ctr_x = deltas[:, 0] * anchors_wh[:, 0] + anchors_ctr[:, 0]
        pred_ctr_y = deltas[:, 1] * anchors_wh[:, 1] + anchors_ctr[:, 1]
        pred_w = torch.exp(deltas[:, 2]) * anchors_wh[:, 0]
        pred_h = torch.exp(deltas[:, 3]) * anchors_wh[:, 1]
        x1 = pred_ctr_x - 0.5 * pred_w
        y1 = pred_ctr_y - 0.5 * pred_h
        x2 = pred_ctr_x + 0.5 * pred_w
        y2 = pred_ctr_y + 0.5 * pred_h
        return torch.stack([x1, y1, x2, y2], dim=1)

    def forward(self, images, targets=None):
        features = self.backbone(images)
        x = F.relu(self.conv_head(features))
        cls_logits = self.cls_logits(x)
        bbox_deltas = self.bbox_pred(x)
        batch_size = images.size(0)
        cls_logits = cls_logits.permute(0, 2, 3, 1).contiguous().view(batch_size, -1, self.num_classes)
        bbox_deltas = bbox_deltas.permute(0, 2, 3, 1).contiguous().view(batch_size, -1, 4)
        if self.training:
            if targets is None: raise ValueError("Targets needed for training.")
            return {'loss': self._compute_loss(cls_logits, bbox_deltas, targets)}
        return self._generate_proposals(cls_logits, bbox_deltas)

    def _compute_loss(self, cls_logits, bbox_deltas, targets):
        batch_size = cls_logits.size(0)
        anchors = self.anchors
        all_cls_targets, all_reg_targets, all_reg_masks = [], [], []
        for i in range(batch_size):
            gt_boxes = targets[i]['boxes']
            gt_labels = targets[i]['labels']
            num_anchors = anchors.size(0)
            if gt_boxes.numel() == 0:
                cls_target = torch.full((num_anchors,), 0, dtype=torch.long, device=cls_logits.device)
                reg_target = torch.zeros((num_anchors, 4), dtype=torch.float, device=cls_logits.device)
                reg_mask = torch.zeros((num_anchors,), dtype=torch.bool, device=cls_logits.device)
            else:
                iou = box_iou(anchors, gt_boxes)
                max_iou, max_iou_indices = iou.max(dim=1)
                cls_target = torch.full((num_anchors,), -1, dtype=torch.long, device=cls_logits.device)
                reg_target = torch.zeros((num_anchors, 4), dtype=torch.float, device=cls_logits.device)
                cls_target[max_iou < self.neg_thresh] = 0
                pos_mask = max_iou >= self.pos_thresh
                cls_target[pos_mask] = gt_labels[max_iou_indices[pos_mask]]
                gt_max_iou, gt_max_iou_indices = iou.max(dim=0)
                if gt_max_iou.numel() > 0:
                    cls_target[gt_max_iou_indices] = gt_labels
                    pos_mask[gt_max_iou_indices] = True
                reg_mask = pos_mask
                if pos_mask.sum() > 0:
                    reg_target[pos_mask] = self._encode_boxes(anchors[pos_mask], gt_boxes[max_iou_indices[pos_mask]])
            all_cls_targets.append(cls_target)
            all_reg_targets.append(reg_target)
            all_reg_masks.append(reg_mask)
        cls_targets = torch.cat(all_cls_targets)
        reg_targets = torch.cat(all_reg_targets)
        reg_masks = torch.cat(all_reg_masks)
        cls_logits = cls_logits.view(-1, self.num_classes)
        bbox_deltas = bbox_deltas.view(-1, 4)
        cls_mask = cls_targets >= 0
        loss_cls = F.cross_entropy(cls_logits[cls_mask], cls_targets[cls_mask], reduction='mean')
        num_pos = reg_masks.sum()
        loss_reg = F.smooth_l1_loss(bbox_deltas[reg_masks], reg_targets[reg_masks], beta=1.0, reduction='sum') / num_pos if num_pos > 0 else torch.tensor(0.0, device=cls_logits.device)
        return loss_cls + loss_reg

    @torch.no_grad()
    def _generate_proposals(self, cls_logits, bbox_deltas, conf_thresh=0.05, nms_thresh=0.45):
        batch_size = cls_logits.size(0)
        anchors = self.anchors
        scores = F.softmax(cls_logits, dim=-1)
        detections = []
        for i in range(batch_size):
            pred_boxes = self._decode_boxes(anchors, bbox_deltas[i])
            scores_fg = scores[i][:, 1:]
            top_scores, top_labels = scores_fg.max(dim=1)
            keep = top_scores > conf_thresh
            boxes_out, scores_out, labels_out = pred_boxes[keep], top_scores[keep], top_labels[keep] + 1
            if boxes_out.numel() == 0:
                detections.append({'boxes': torch.empty(0, 4), 'scores': torch.empty(0), 'labels': torch.empty(0, dtype=torch.long)})
                continue
            keep_nms = batched_nms(boxes_out, scores_out, labels_out, nms_thresh)
            detections.append({'boxes': boxes_out[keep_nms], 'scores': scores_out[keep_nms], 'labels': labels_out[keep_nms]})
        return detections

In [6]:
def compute_iou(box1, box2):
    # Use standard torchvision for speed and correctness if possible
    if isinstance(box1, np.ndarray): box1 = torch.tensor(box1)
    if isinstance(box2, np.ndarray): box2 = torch.tensor(box2)
    return box_iou(box1.unsqueeze(0) if box1.ndim==1 else box1,
                   box2.unsqueeze(0) if box2.ndim==1 else box2).squeeze()

def compute_ap(predictions, ground_truth, iou_threshold=0.5):
    if len(predictions) == 0: return 0.0
    if len(ground_truth) == 0: return 0.0
    predictions = sorted(predictions, key=lambda x: x['score'], reverse=True)
    tp, fp = np.zeros(len(predictions)), np.zeros(len(predictions))
    gt_matched = set()
    for i, pred in enumerate(predictions):
        # Use torchvision box_iou for efficiency
        iou = box_iou(torch.tensor([pred['box']]), torch.tensor(ground_truth)).squeeze(0)
        if iou.numel() > 0:
            best_iou, best_gt_idx = iou.max(dim=0)
            best_iou, best_gt_idx = best_iou.item(), best_gt_idx.item()
        else:
             best_iou, best_gt_idx = 0, -1

        if best_iou >= iou_threshold:
            if best_gt_idx not in gt_matched:
                tp[i] = 1
                gt_matched.add(best_gt_idx)
            else: fp[i] = 1
        else: fp[i] = 1
    tp_cumsum, fp_cumsum = np.cumsum(tp), np.cumsum(fp)
    recalls = tp_cumsum / len(ground_truth)
    precisions = tp_cumsum / (tp_cumsum + fp_cumsum + 1e-6)
    ap = 0
    for t in np.arange(0, 1.1, 0.1):
        ap += (np.max(precisions[recalls >= t]) if np.sum(recalls >= t) > 0 else 0) / 11
    return ap

In [7]:
def train_one_epoch(model, dataloader, optimizer, device, epoch, num_epochs, warmup_epochs, base_lr):
    model.train()
    total_loss = 0
    if epoch < warmup_epochs:
        lr = base_lr * (epoch + 1) / warmup_epochs
        for param_group in optimizer.param_groups: param_group['lr'] = lr
    else: lr = optimizer.param_groups[0]['lr']

    loader = tqdm(dataloader, desc=f"Epoch {epoch}/{num_epochs} [Train]")
    for images, targets in loader:
        images = images.to(device)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        optimizer.zero_grad()
        loss = model(images, targets)['loss']
        if not torch.isfinite(loss): continue
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        loader.set_postfix(loss=loss.item(), lr=lr)
    return total_loss / len(dataloader)

@torch.no_grad()
def evaluate(model, dataloader, device, num_fg_classes):
    total_val_loss = 0
    all_preds_by_class = defaultdict(list)
    all_gts_by_class = defaultdict(list)
    ious_for_miou = [] # List to store IoUs of matched detections

    loader = tqdm(dataloader, desc="Evaluating")
    for images, targets in loader:
        images = images.to(device)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # 1. Get Loss
        model.train()
        total_val_loss += model(images, targets)['loss'].item()

        # 2. Get Predictions
        model.eval()
        predictions = model(images)

        for i, (pred_dict, gt_dict) in enumerate(zip(predictions, targets)):
            # --- mIoU Calculation ---
            if len(pred_dict['boxes']) > 0 and len(gt_dict['boxes']) > 0:
                 # Calculate IoU for ALL predictions against ALL GTs in this image
                 iou_matrix = box_iou(pred_dict['boxes'], gt_dict['boxes'])
                 # For each prediction, find its BEST matching GT
                 max_ious, _ = iou_matrix.max(dim=1)
                 # Only consider "reasonable" detections (e.g. IoU > 0.5) for the Average IoU metric
                 # to avoid dragging down the score with obvious false positives.
                 # This gives "Average IoU of True Positives".
                 valid_ious = max_ious[max_ious > 0.5]
                 if valid_ious.numel() > 0:
                     ious_for_miou.extend(valid_ious.cpu().tolist())

            # --- Data for mAP ---
            for cls_id in range(1, num_fg_classes + 1):
                gt_mask = gt_dict['labels'] == cls_id
                all_gts_by_class[cls_id].extend(gt_dict['boxes'][gt_mask].cpu())
                pred_mask = pred_dict['labels'] == cls_id
                for box, score in zip(pred_dict['boxes'][pred_mask], pred_dict['scores'][pred_mask]):
                    all_preds_by_class[cls_id].append({'box': box.cpu().numpy(), 'score': score.cpu().item()})

    ap_per_class = [compute_ap(all_preds_by_class[c], np.array(all_gts_by_class[c])) for c in range(1, num_fg_classes + 1)]
    # Calculate mean of collected IoUs
    mIoU = np.mean(ious_for_miou) if len(ious_for_miou) > 0 else 0.0

    return total_val_loss / len(dataloader), np.mean(ap_per_class), mIoU

In [8]:
class TrainingTracker:
    def __init__(self):
        self.history = {'train_loss': [], 'val_loss': [], 'val_mAP': [], 'val_mIoU': [], 'learning_rate': []}
        self.best_val_mAP = 0.0
        self.patience_counter = 0

    def update(self, epoch, train_loss, val_loss, val_mAP, val_mIoU, lr):
        self.history['train_loss'].append(train_loss)
        self.history['val_loss'].append(val_loss)
        self.history['val_mAP'].append(val_mAP)
        self.history['val_mIoU'].append(val_mIoU)
        self.history['learning_rate'].append(lr)
        if val_mAP > self.best_val_mAP:
            self.best_val_mAP = val_mAP
            self.patience_counter = 0
            return 0, True
        self.patience_counter += 1
        return self.patience_counter, False

    def print_epoch(self, epoch, train_loss, val_loss, val_mAP, val_mIoU, lr):
        print(f"Epoch {epoch:3d} | TrnLoss: {train_loss:.4f} | ValLoss: {val_loss:.4f} | "
              f"mAP: {val_mAP:.4f} | mIoU: {val_mIoU:.4f} | LR: {lr:.2e}")

In [9]:
class Trainer:
    def __init__(self, config):
        self.config = config
        self.device = torch.device(config['training']['device'])
        self.tracker = TrainingTracker()
        self.model = ObjectDetectorFromScratch(
            num_fg_classes=config['model']['num_fg_classes'],
            num_anchors=config['model']['num_anchors'],
            target_size=config['model']['image_size']
        ).to(self.device)
        self.train_loader, self.val_loader = self._setup_dataloaders()
        self.optimizer = optim.AdamW(self.model.parameters(), lr=config['training']['learning_rate'], weight_decay=config['training']['weight_decay'])
        self.scheduler = CosineAnnealingLR(self.optimizer, T_max=config['training']['num_epochs'] - config['training']['warmup_epochs'], eta_min=1e-6)

    def _setup_dataloaders(self):
        train_tf = FloorplanAugmentation(train=True, target_size=self.config['model']['image_size'])
        val_tf = FloorplanAugmentation(train=False, target_size=self.config['model']['image_size'])
        train_ds = COCOFloorplanDataset(self.config['dataset']['image_dir'], self.config['dataset']['annotation_file'], train_tf)
        val_ds = COCOFloorplanDataset(self.config['dataset']['image_dir'], self.config['dataset']['annotation_file'], val_tf)
        indices = list(range(len(train_ds)))
        random.shuffle(indices)
        split = int(len(train_ds) * self.config['dataset']['train_split'])
        print(f"Dataset split: {split} train, {len(indices) - split} val")
        return (DataLoader(Subset(train_ds, indices[:split]), batch_size=self.config['training']['batch_size'], shuffle=True, num_workers=2, collate_fn=collate_fn, pin_memory=True),
                DataLoader(Subset(val_ds, indices[split:]), batch_size=self.config['training']['batch_size'], shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True))

    def train(self):
        print(f"\nSTARTING TRAINING | Device: {self.device} | Params: {sum(p.numel() for p in self.model.parameters() if p.requires_grad):,}")
        cfg = self.config['training']
        EVAL_EVERY = 5
        for epoch in range(cfg['num_epochs']):
            train_loss = train_one_epoch(self.model, self.train_loader, self.optimizer, self.device, epoch, cfg['num_epochs'], cfg['warmup_epochs'], cfg['learning_rate'])
            if (epoch + 1) % EVAL_EVERY == 0 or epoch == cfg['num_epochs'] - 1:
                val_loss, val_mAP, val_mIoU = evaluate(self.model, self.val_loader, self.device, self.config['model']['num_fg_classes'])
                lr = self.optimizer.param_groups[0]['lr']
                patience, is_best = self.tracker.update(epoch, train_loss, val_loss, val_mAP, val_mIoU, lr)
                self.tracker.print_epoch(epoch, train_loss, val_loss, val_mAP, val_mIoU, lr)
                if is_best:
                    torch.save(self.model.state_dict(), 'best_model.pth')
                    print(f"  -> New best model (mAP: {val_mAP:.4f}, mIoU: {val_mIoU:.4f})")
                if patience >= cfg['early_stopping_patience']:
                    print(f"Early stopping at epoch {epoch}"); break
            else: print(f"Epoch {epoch:3d} | Train Loss: {train_loss:.4f}")
            if epoch >= cfg['warmup_epochs']: self.scheduler.step()
        torch.save(self.model.state_dict(), 'final_model.pth')
        return self.tracker.history

def main():
    if CONFIG['training']['device'] == 'cpu': print("WARNING: Training on CPU will be slow.")
    if not Path(CONFIG['dataset']['image_dir']).exists() or not Path(CONFIG['dataset']['annotation_file']).exists():
        print("Error: Dataset not found. Check paths in CONFIG."); return
    trainer = Trainer(CONFIG)
    trainer.train()

if __name__ == "__main__":
    main()

Dataset split: 436 train, 109 val

STARTING TRAINING | Device: cuda | Params: 1,812,085


Epoch 0/50 [Train]: 100%|██████████| 109/109 [01:38<00:00,  1.11it/s, loss=0.721, lr=2e-5]


Epoch   0 | Train Loss: 1.5445


Epoch 1/50 [Train]: 100%|██████████| 109/109 [00:08<00:00, 12.28it/s, loss=0.204, lr=4e-5]


Epoch   1 | Train Loss: 0.3124


...

Epoch  47 | Train Loss: 0.0277


Epoch 48/50 [Train]: 100%|██████████| 109/109 [00:09<00:00, 11.94it/s, loss=0.0388, lr=1.48e-6]


Epoch  48 | Train Loss: 0.0274


Evaluating: 100%|██████████| 28/28 [00:02<00:00, 11.36it/s]


Epoch  49 | TrnLoss: 0.0276 | ValLoss: 0.0741 | mAP: 0.1187 | mIoU: 0.6853 | LR: 1.12e-06
  -> New best model (mAP: 0.1187, mIoU: 0.6853)
